In [3]:
import requests
import pandas as pd
import boto3
from datetime import datetime as dt
from datetime import timedelta as td



In [2]:
!aws s3 ls

2023-08-16 22:50:34 aigithubs
2023-08-08 20:46:37 dttghaactivity
2022-11-23 17:38:26 elasticbeanstalk-us-east-1-130663382464
2022-11-23 22:00:09 sadfdsfasdfasdfasfdsafsd


In [4]:
!aws s3 rm s3://aigithubs --recursive

In [5]:
!aws s3 mb s3://aigithubs

make_bucket: aigithubs


In [8]:

dynamodb = boto3.resource('dynamodb')
table= dynamodb.Table('jobs')
table.get_item(Key={'job_id':'ghactivity_ingest'})
item = {
     'job_id': 'ghactivity_ingest',
     'job_description': 'Ingest ghactivity data to s3',
     'is_active': 'Y',
     'baseline_days': 3,
}

table.put_item(Item=item)

    
      

{'ResponseMetadata': {'RequestId': 'BVFFF3GKMG141632KKS1PD696RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 18 Aug 2023 11:48:21 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'BVFFF3GKMG141632KKS1PD696RVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [9]:
baseline_days = table.\
    get_item(Key={'job_id':'ghactivity_ingest'})['Item']['baseline_days']
baseline_days

Decimal('3')

In [10]:
int(baseline_days)

3

In [11]:

next_file = f"{dt.strftime(dt.now().date()- td(days=int(baseline_days)),'%Y-%m-%d-%-H')}.json.gz"
next_file

'2023-08-15-0.json.gz'

In [14]:
res = requests.get(f'https://data.gharchive.org/{next_file}')


In [12]:
s3_client = boto3.client('s3')

In [20]:
bucket_name = 'aigithubs'

upload_res = s3_client.put_object(
    Bucket=bucket_name,
    Key=f'landing/ghactivity/{next_file}',
    Body=res.content
)

In [21]:
upload_res

{'ResponseMetadata': {'RequestId': '3KMEFZNS2RSKENDS',
  'HostId': 'v+4FaADjKVEWzFb0/CgMKEnqaxHRUauRPBrkhTf5KHmE/GViHODs90pCTaK4BkQN7TfdTHkP/fo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'v+4FaADjKVEWzFb0/CgMKEnqaxHRUauRPBrkhTf5KHmE/GViHODs90pCTaK4BkQN7TfdTHkP/fo=',
   'x-amz-request-id': '3KMEFZNS2RSKENDS',
   'date': 'Fri, 18 Aug 2023 11:05:01 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3ffe98b7af5cd693579cfc8c1e098e84"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3ffe98b7af5cd693579cfc8c1e098e84"',
 'ServerSideEncryption': 'AES256'}

In [13]:
bucket_name = 'aigithubs'
df = pd.read_json(f's3://{bucket_name}/landing/ghactivity/{next_file}',
                lines=True,
                orient='records')


In [14]:
df.shape

(164736, 8)

In [15]:
df.count()

id            164736
type          164736
actor         164736
repo          164736
payload       164736
public        164736
created_at    164736
org            36344
dtype: int64

In [16]:
df.head()

,id,type,actor,repo,payload,public,created_at,org
0,31116151901,PushEvent,"{'id': 73243839, 'login': 'akum2', 'display_lo...","{'id': 348002433, 'name': 'akum2/akum2', 'url'...","{'repository_id': 348002433, 'push_id': 146751...",True,2023-08-15 00:00:00+00:00,NaN
1,31116151908,PushEvent,"{'id': 98237115, 'login': 'Daniil-Horobets', '...","{'id': 678423192, 'name': 'Daniil-Horobets/Ada...","{'repository_id': 678423192, 'push_id': 146751...",True,2023-08-15 00:00:00+00:00,NaN
2,31116151909,PushEvent,"{'id': 1737202, 'login': 'rsathish', 'display_...","{'id': 678595806, 'name': 'rsathish/Niraltek',...","{'repository_id': 678595806, 'push_id': 146751...",True,2023-08-15 00:00:00+00:00,NaN
3,31116151914,WatchEvent,"{'id': 87281783, 'login': 'malisipi', 'display...","{'id': 166474509, 'name': 'glreno/oneko', 'url...",{'action': 'started'},True,2023-08-15 00:00:00+00:00,NaN
4,31116151925,PushEvent,"{'id': 109624739, 'login': 'shopify-github-act...","{'id': 88897663, 'name': 'Shopify/polaris', 'u...","{'repository_id': 88897663, 'push_id': 1467516...",True,2023-08-15 00:00:00+00:00,"{'id': 8085, 'login': 'Shopify', 'gravatar_id'..."
